1.

In [1]:
import pymysql
from sqlalchemy import create_engine
import pandas as pd
import getpass # To get the password without showing the input
from sqlalchemy import text

password = getpass.getpass("Please enter the SQL database password: ")
connection_string = 'mysql+pymysql://root:' + password + '@localhost/sakila'
engine = create_engine(connection_string)

2.

In [13]:
def rentals_month(engine, month, year):
    
    query = "SELECT * FROM rental WHERE MONTH(rental_date) = "+str(month)+" AND YEAR(rental_date) = "+str(year)
    
    with engine.connect() as connection:    
        query = text(query)
        result = connection.execute(query)
        df = pd.DataFrame(result.all())
    
    return df

In [14]:
mayo = rentals_month(engine, 5, 2005)
mayo.head(1)

,rental_id,rental_date,inventory_id,customer_id,return_date,staff_id,last_update
0,1,2005-05-24 22:53:30,367,130,2005-05-26 22:04:30,1,2006-02-15 21:30:53


3.

In [15]:
def rental_count_month(df, month, year):

    rentals_by_customer = df.loc[(df['rental_date'].dt.month == month) & (df['rental_date'].dt.year == year)]\
                          .groupby('customer_id')['rental_id'].count().reset_index()

    # rename the 'rental_id' column to 'rentals_{month}'
    rentals_by_customer = rentals_by_customer.rename(columns={'rental_id': f'rentals_{month}_{year}'})

    return rentals_by_customer

In [16]:
customer_rentals_mayo = rental_count_month(mayo, 5, 2005)
customer_rentals_mayo

,customer_id,rentals_5_2005
0,1,2
1,2,1
2,3,2
3,5,3
4,6,3
...,...,...
515,594,4
516,595,1
517,596,6
518,597,2


4.

In [17]:
def compare_rentals(df1, df2):
    
    merged_df = pd.merge(df1, df2, on='customer_id')

    merged_df['difference'] = merged_df[df1.columns[1]] - merged_df[df2.columns[1]]
    
    return merged_df

In [18]:
junio = rentals_month(engine,6, 2005)
customer_rentals_junio = rental_count_month(junio, 6, 2005)
customer_rentals_junio

comparative = compare_rentals(customer_rentals_mayo, customer_rentals_junio)
comparative

,customer_id,rentals_5_2005,rentals_6_2005,difference
0,1,2,7,-5
1,2,1,1,0
2,3,2,4,-2
3,5,3,5,-2
4,6,3,4,-1
...,...,...,...,...
507,594,4,6,-2
508,595,1,2,-1
509,596,6,2,4
510,597,2,3,-1
